In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

# RNN介绍

| x1  | x2  | x3  | y |
| --- | --- | --- | - |
| 001 | 010 | 100 | 1 |
| 100 | 111 | 000 | 0 |
| 001 | 010 | 100 | 1 |
| 100 | 000 | 111 | 1 |
| 111 | 010 | 100 | 0 |
| 100 | 010 | 000 | 0 |

x1,x2,x3就是$\color{#FF0000}{seq len}$序列的长度，每个下面由一个长度为3的向量组成，这个向量就是$\color{#FF0000}{input size}$。  
对于语句一行数据就是一句话，每句话有三个单词组成，每个单词用001这样的one-hot编码。  

![avatar](image/rnn.jpg)

![avatar](image/rnn_nlp.png)

# 建立输入到输出

out输出的是所有h1到h_N的隐层结果，hidden输出的就是最后一次计算后的结果值。这里N个RNN层其实就是同一个

batch_size是输入多少条数据  
seq_len是循环的长度，比如根据前n天天气的n，前n天股票信息的n  
input_size是每一天的信息维度  
hidden_size是输出的数据维度

In [35]:
batch_size = 5
seq_len = 3
input_size = 4
hidden_size = 2
num_layers = 1

In [39]:
cell = nn.RNN(input_size = input_size,hidden_size = hidden_size,num_layers = num_layers,batch_first=True)

batch_first设置为True后，输入数据的要把batch_size放在第一维

In [40]:
inputs = torch.ones(batch_size,seq_len,input_size)
hidden = torch.zeros(num_layers,batch_size,hidden_size)

In [41]:
inputs.shape,hidden.shape

(torch.Size([5, 3, 4]), torch.Size([1, 5, 2]))

In [45]:
inputs[0]

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

第一个数据的维度是3，4

In [43]:
out,hidden = cell(inputs,hidden)
out.shape,hidden.shape

(torch.Size([5, 3, 2]), torch.Size([1, 5, 2]))

In [44]:
out,hidden

(tensor([[[-0.7220,  0.9815],
          [-0.3707,  0.9855],
          [-0.4992,  0.9907]],
 
         [[-0.7220,  0.9815],
          [-0.3707,  0.9855],
          [-0.4992,  0.9907]],
 
         [[-0.7220,  0.9815],
          [-0.3707,  0.9855],
          [-0.4992,  0.9907]],
 
         [[-0.7220,  0.9815],
          [-0.3707,  0.9855],
          [-0.4992,  0.9907]],
 
         [[-0.7220,  0.9815],
          [-0.3707,  0.9855],
          [-0.4992,  0.9907]]], grad_fn=<TransposeBackward1>),
 tensor([[[-0.4992,  0.9907],
          [-0.4992,  0.9907],
          [-0.4992,  0.9907],
          [-0.4992,  0.9907],
          [-0.4992,  0.9907]]], grad_fn=<StackBackward>))